# Sampling RDDs

So far we have introduced RDD creation together with some basic transformations such as `map` and `filter` and some actions such as `count`, `take`, and `collect`.  

This notebook will show how to sample RDDs. Regarding transformations, `sample` will be introduced since it will be useful in many statistical learning scenarios. Then we will compare results with the `takeSample` action.      

## Getting the data and creating the RDD

In this case we will use the complete dataset provided for the KDD Cup 1999, containing nearly half million network interactions. The file is provided as a Gzip file that we will download locally.

In [11]:
# 为了能够让 python 找到 pyspark，使用 findspark
import findspark
findspark.init()

# 为了使用 RDDs，创建 SparkSession
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# 创建 SparkConf 和 SparkSession
conf=SparkConf()\
        .setMaster('local[*]')\
        .setAppName("~")\
        .setExecutorEnv("spark.executor.memory","2g")\
        .setExecutorEnv("spark.driver.memory","2g")

spark=SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

# 获取 SparkContext
sc=spark.sparkContext

In [12]:
import urllib
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")

KeyboardInterrupt: 

Now we can use this file to create our RDD.

In [2]:
!hadoop fs -put ./kddcup.data.gz /kddcup.data.gz

put: `/kddcup.data.gz': File exists


In [13]:
data_file = "/kddcup.data.gz"
raw_data = sc.textFile(data_file)

## Sampling RDDs   

In Spark, there are two sampling operations, the transformation `sample` and the action `takeSample`. By using a transformation we can tell Spark to apply successive transformation on a sample of a given RDD. By using an action we retrieve a given sample and we can have it in local memory to be used by any other standard library (e.g. Scikit-learn).  

![](asset/sample-size-definition.png)

### The `sample` transformation

The `sample` transformation takes up to three parameters.
- First is whether the sampling is done with replacement or not. 
- Second is the sample size as a fraction. 
- Finally we can optionally provide a **random seed**.  

In [15]:
raw_data_sample = raw_data.sample(False, 0.01, 1234)
raw_data_sample

PythonRDD[13] at RDD at PythonRDD.scala:53

In [17]:
sample_size = raw_data_sample.count()
sample_size

49085

In [18]:
total_size = raw_data.count()
print("Sample size is {} of {}".format(sample_size, total_size))

Sample size is 49085 of 4898431


But the power of sampling as a transformation comes from doing it as part of a sequence of additional transformations. This will show more powerful once we start doing aggregations and key-value pairs operations, and will be specially useful when using Spark's machine learning library MLlib.    

In the meantime, imagine we want to have an approximation of the proportion of `normal.` interactions in our dataset. We could do this by counting the total number of tags as we did in previous notebooks. However we want a quicker response and we don't need the exact answer but just an approximation. We can do it as follows.   

In [19]:
from time import time

# transformations to be applied
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("The ratio of 'normal' interactions is {}".format(round(sample_normal_ratio,3)))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.197
Count done in 17.277 seconds


Let's compare this with calculating the ratio without sampling.  

In [7]:
# transformations to be applied
raw_data_items = raw_data.map(lambda x: x.split(","))
normal_tags = raw_data_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
normal_tags_count = normal_tags.count()
tt = time() - t0

normal_ratio = normal_tags_count / float(total_size)
print("The ratio of 'normal' interactions is {}".format(round(normal_ratio,3)))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.199
Count done in 35.767 seconds


We can see a gain in time. The more transformations we apply after the sampling the bigger this gain. This is because without sampling all the transformations are applied to the complete set of data.  

### The `takeSample` action  

If what we need is to grab a sample of raw data from our RDD into local memory in order to be used by other non-Spark libraries, `takeSample` can be used.  

The syntax is very similar, but in this case we specify the number of items instead of the sample size as a fraction of the complete data size.  

In [10]:
t0 = time()
raw_data_sample = raw_data.takeSample(False, 4000, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt = time() - t0

normal_sample_size = len(normal_data_sample)

normal_ratio = normal_sample_size / 400000.0
print("The ratio of 'normal' interactions is {}".format(normal_ratio))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.0019675
Count done in 30.338 seconds


The process was very similar as before. We obtained a sample of about 10 percent of the data, and then filter and split.  

However, it took longer, even with a slightly smaller sample. The reason is that Spark just distributed the execution of the sampling process. The filtering and splitting of the results were done locally in a single node.  